# On-Demand Sleep-Based Aloha Simulator**Final Year Project**  **Date:** February 2026## OverviewComplete implementation of a discrete-event simulator for M2M communication using Slotted Aloha with on-demand sleep.### Features- Slotted Aloha with collision detection- On-demand sleep mechanism- Energy tracking (4 power states)- Performance metrics (lifetime, delay, throughput)- Parameter optimization- Visualization### How to Use This Notebook1. Run cells sequentially from top to bottom2. Edit configuration in Section 23. View results and plots4. Experiment with parameters**All code is self-contained in this notebook!**

## 1. Import Dependencies

In [ ]:
# Install if needed (uncomment):# !pip install simpy numpy matplotlib seaborn pandasimport simpyimport randomimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsimport pandas as pdfrom collections import dequefrom dataclasses import dataclassfrom typing import List, Dict, Optionalsns.set_style("whitegrid")plt.rcParams['figure.figsize'] = (12, 6)print("Dependencies loaded!")

## 2. Configuration Parameters**Edit these to customize your simulation:**

In [ ]:
# ==================================================# SIMULATION CONFIGURATION - Edit These Values# ==================================================# System ParametersN_NODES = 10              # Number of nodesLAMBDA_ARRIVAL = 0.1      # Packet arrival rate (λ)Q_TRANSMIT = 0.05         # Transmission probability (q)TS_IDLE = 5               # Idle timeout (slots)TW_WAKEUP = 2             # Wake-up duration (slots)# Energy Parameters  E_INITIAL = 10000         # Initial energyPS_SLEEP = 0.1            # Sleep powerPW_WAKEUP = 1.0           # Wake-up powerPT_TRANSMIT = 5.0         # Transmit powerPB_BUSY = 0.5             # Busy power# SimulationSIMULATION_TIME = 5000    # Duration (slots)RANDOM_SEED = 42          # For reproducibilityprint("="*70)print(f"Configuration: {N_NODES} nodes, λ={LAMBDA_ARRIVAL}, q={Q_TRANSMIT}")print(f"Sleep: ts={TS_IDLE}, tw={TW_WAKEUP}, E={E_INITIAL}")print(f"Duration: {SIMULATION_TIME} slots")print("="*70)

## 3. Core Simulator Classes### 3.1 Packet Class

In [ ]:
@dataclassclass Packet:    """Data packet"""    size: int    src: str    dest: str    arrival_time: float    packet_id: int = 0        def __repr__(self):        return f"Packet(id={self.packet_id}, src={self.src})"print("Packet class defined")

### 3.2 Node Class (MTD)Implements 4-state machine: active → idle → sleep → wake-up

In [ ]:
"""    Represents a Machine-Type Device (MTD) in the M2M network.        States:    - active: Node has packets and is actively trying to transmit    - idle: Buffer is empty, waiting for ts slots before sleeping    - sleep: Deep sleep mode (very low power)    - wakeup: Transitioning from sleep to active (takes tw slots)    """        def __init__(self, env: simpy.Environment, name: str,                  lambda_arrival: float, q_transmit: float,                  ts_idle: int, tw_wakeup: int, E_initial: float,                 PS_sleep: float, PW_wakeup: float,                  PT_transmit: float, PB_busy: float,                 simulator=None):        # SimPy environment        self.env = env        self.name = name        self.simulator = simulator                # Buffer and state        self.buffer = deque()  # Infinite queue for packets        self.state = 'active'  # Initial state        self.idle_timer = 0  # Countdown timer for idle->sleep transition        self.wakeup_timer = 0  # Countdown timer for wakeup duration                # Protocol parameters        self.lambda_arrival = lambda_arrival  # Bernoulli arrival rate (prob per slot)        self.q_transmit = q_transmit  # Transmission probability per slot        self.ts_idle = ts_idle  # Idle timeout (slots before sleep)        self.tw_wakeup = tw_wakeup  # Wake-up duration (slots)                # Energy parameters        self.E_initial = E_initial        self.energy = E_initial  # Current remaining energy        self.PS = PS_sleep  # Power consumption in sleep        self.PW = PW_wakeup  # Power consumption during wake-up        self.PT = PT_transmit  # Power consumption when transmitting        self.PB = PB_busy  # Power consumption when active/idle but not transmitting                # Metrics tracking        self.packets_sent = 0  # Successfully transmitted packets        self.packets_arrived = 0  # Total packets arrived        self.total_delay = 0.0  # Sum of queueing delays        self.lifetime = None  # Time when energy depleted        self.total_transmit_attempts = 0  # Total transmission attempts        self.collisions = 0  # Number of collisions experienced        self.packet_id_counter = 0  # For unique packet IDs                # State history (for debugging/analysis)        self.state_history = []        self.energy_history = []            def generate_traffic(self):        """        Traffic generation process: Bernoulli arrivals.        In each slot, a new packet arrives with probability lambda_arrival.        """        while self.energy > 0:            # Bernoulli arrival            if random.random() < self.lambda_arrival:                packet = Packet(                    size=1000,  # Fixed packet size                    src=self.name,                    dest='receiver',                    arrival_time=self.env.now,                    packet_id=self.packet_id_counter                )                self.packet_id_counter += 1                self.packets_arrived += 1                self.buffer.append(packet)                                # If sleeping, trigger wake-up                if self.state == 'sleep':                    self.start_wakeup()                # If idle, cancel idle timer and become active                elif self.state == 'idle':                    self.state = 'active'                    self.idle_timer = 0                                yield self.env.timeout(1)  # Advance one slot                def start_wakeup(self):        """Initiate wake-up transition from sleep"""        self.state = 'wakeup'        self.wakeup_timer = self.tw_wakeup            def process_slot(self):        """        Main slot processing loop.        Called every slot to handle state transitions, transmissions, and energy consumption.        """        while True:            # Check if energy depleted            if self.energy <= 0:                if self.lifetime is None:                    self.lifetime = self.env.now                    self.state = 'dead'                yield self.env.timeout(1)                continue                            # Record state for history            self.state_history.append((self.env.now, self.state))                        power_consumed = 0            will_transmit = False                        # State machine            if self.state == 'sleep':                power_consumed = self.PS                # Wake-up is triggered by packet arrival (in generate_traffic)                            elif self.state == 'wakeup':                power_consumed = self.PW                self.wakeup_timer -= 1                if self.wakeup_timer <= 0:                    self.state = 'active'                                elif self.state == 'active':                if len(self.buffer) > 0:                    # Node has packets: decide whether to transmit                    if random.random() < self.q_transmit:                        power_consumed = self.PT                        will_transmit = True                    else:                        power_consumed = self.PB  # Active but not transmitting                else:                    # Buffer empty: transition to idle                    self.state = 'idle'                    self.idle_timer = self.ts_idle                    power_consumed = self.PB                                elif self.state == 'idle':                power_consumed = self.PB                if len(self.buffer) > 0:                    # New packet arrived: back to active                    self.state = 'active'                    self.idle_timer = 0                else:                    # Countdown idle timer                    self.idle_timer -= 1                    if self.idle_timer <= 0:                        self.state = 'sleep'                                    # Consume energy            self.energy -= power_consumed            self.energy_history.append((self.env.now, self.energy))                        # Register transmission attempt with simulator            if will_transmit and len(self.buffer) > 0:                self.total_transmit_attempts += 1                self.simulator.register_transmission(self)                            yield self.env.timeout(1)  # Advance one slot                def handle_transmission_result(self, success: bool):        """        Called by simulator to notify transmission result.        If successful, remove HOL packet from buffer and record delay.        """        if success and len(self.buffer) > 0:            sent_packet = self.buffer.popleft()            delay = self.env.now - sent_packet.arrival_time            self.total_delay += delay            self.packets_sent += 1        elif not success:            self.collisions += 1                def get_avg_delay(self) -> float:        """Calculate average queueing delay"""        return self.total_delay / self.packets_sent if self.packets_sent > 0 else 0        def get_throughput(self, sim_time: float) -> float:        """Calculate throughput (packets/slot)"""        return self.packets_sent / sim_time if sim_time > 0 else 0

### 3.3 Simulator ClassCoordinates all nodes and handles collision detection.

In [ ]:
class Simulator:"""    Main simulator class that manages all nodes and the shared channel.    Handles collision detection for slotted Aloha.    """        def __init__(self, env: simpy.Environment, n_nodes: int,                 lambda_arrival: float, q_transmit: float,                 ts_idle: int, tw_wakeup: int, E_initial: float,                 PS: float, PW: float, PT: float, PB: float,                 simulation_time: int, seed: Optional[int] = None):        self.env = env        self.simulation_time = simulation_time                # Set random seed for reproducibility        if seed is not None:            random.seed(seed)            np.random.seed(seed)                    # Create nodes        self.nodes = []        for i in range(n_nodes):            node = Node(                env=env,                name=f'Node_{i}',                lambda_arrival=lambda_arrival,                q_transmit=q_transmit,                ts_idle=ts_idle,                tw_wakeup=tw_wakeup,                E_initial=E_initial,                PS_sleep=PS,                PW_wakeup=PW,                PT_transmit=PT,                PB_busy=PB,                simulator=self            )            self.nodes.append(node)                    # Per-slot transmission tracking        self.transmitting_nodes_current_slot = []                # Global metrics        self.total_slots_simulated = 0        self.total_successful_transmissions = 0        self.total_collisions = 0            def register_transmission(self, node: Node):        """Called by nodes when they attempt transmission in current slot"""        self.transmitting_nodes_current_slot.append(node)            def slot_coordinator(self):        """        Global slot coordinator that runs at the end of each slot        to resolve collisions and notify nodes of results.        """        while True:            yield self.env.timeout(0.99)  # Run just before slot boundary                        # Resolve transmissions for this slot            num_transmitters = len(self.transmitting_nodes_current_slot)                        if num_transmitters == 1:                # Success: exactly one transmitter                self.transmitting_nodes_current_slot[0].handle_transmission_result(True)                self.total_successful_transmissions += 1            elif num_transmitters > 1:                # Collision: multiple transmitters                for node in self.transmitting_nodes_current_slot:                    node.handle_transmission_result(False)                self.total_collisions += 1                            # Clear for next slot            self.transmitting_nodes_current_slot = []            self.total_slots_simulated += 1                        yield self.env.timeout(0.01)  # Complete the slot                def run(self):        """Start all processes and run simulation"""        # Start processes for each node        for node in self.nodes:            self.env.process(node.generate_traffic())            self.env.process(node.process_slot())                    # Start slot coordinator        self.env.process(self.slot_coordinator())                # Run simulation        self.env.run(until=self.simulation_time)            def collect_metrics(self) -> Dict:        """        Collect and aggregate metrics from all nodes after simulation.        Returns dictionary with performance metrics.        """        # Per-node metrics        lifetimes = [node.lifetime if node.lifetime else self.simulation_time                      for node in self.nodes]        delays = [node.get_avg_delay() for node in self.nodes]        throughputs = [node.get_throughput(self.simulation_time) for node in self.nodes]                # Aggregate metrics        total_packets_sent = sum(node.packets_sent for node in self.nodes)        total_packets_arrived = sum(node.packets_arrived for node in self.nodes)        total_attempts = sum(node.total_transmit_attempts for node in self.nodes)        total_node_collisions = sum(node.collisions for node in self.nodes)                # Energy metrics        avg_energy_remaining = np.mean([node.energy for node in self.nodes])        energy_consumed_ratio = 1 - (avg_energy_remaining / self.nodes[0].E_initial)                metrics = {            # Lifetime metrics            'avg_lifetime': np.mean(lifetimes),            'min_lifetime': np.min(lifetimes),            'max_lifetime': np.max(lifetimes),            'std_lifetime': np.std(lifetimes),                        # Delay metrics            'avg_delay': np.mean([d for d in delays if d > 0]),            'max_delay': np.max([d for d in delays if d > 0]) if any(d > 0 for d in delays) else 0,                        # Throughput metrics            'total_throughput': total_packets_sent / self.simulation_time,            'avg_node_throughput': np.mean(throughputs),                        # Transmission statistics            'total_packets_sent': total_packets_sent,            'total_packets_arrived': total_packets_arrived,            'packet_delivery_ratio': total_packets_sent / total_packets_arrived if total_packets_arrived > 0 else 0,            'total_transmission_attempts': total_attempts,            'total_collisions': self.total_collisions,            'collision_rate': self.total_collisions / total_attempts if total_attempts > 0 else 0,                        # Energy metrics            'avg_energy_remaining': avg_energy_remaining,            'energy_consumed_ratio': energy_consumed_ratio,                        # System metrics            'n_nodes': len(self.nodes),            'simulation_time': self.simulation_time,            'lambda_arrival': self.nodes[0].lambda_arrival,            'q_transmit': self.nodes[0].q_transmit,            'ts_idle': self.nodes[0].ts_idle,            'tw_wakeup': self.nodes[0].tw_wakeup,        }                return metrics        def get_detailed_node_metrics(self) -> List[Dict]:        """Get detailed metrics for each individual node"""        node_metrics = []        for node in self.nodes:            metrics = {                'name': node.name,                'lifetime': node.lifetime if node.lifetime else self.simulation_time,                'packets_sent': node.packets_sent,                'packets_arrived': node.packets_arrived,                'avg_delay': node.get_avg_delay(),                'throughput': node.get_throughput(self.simulation_time),                'energy_remaining': node.energy,                'total_attempts': node.total_transmit_attempts,                'collisions': node.collisions,                'final_state': node.state,                'buffer_size': len(node.buffer),            }            node_metrics.append(metrics)        return node_metricsprint("Simulator class defined")

## 4. Visualization Functions

In [ ]:
def plot_results(metrics):    """Plot simulation results"""    fig, axes = plt.subplots(2, 3, figsize=(15, 10))    fig.suptitle('Simulation Results', fontsize=16, fontweight='bold')        # Lifetime    ax = axes[0, 0]    data = [metrics['min_lifetime'], metrics['avg_lifetime'], metrics['max_lifetime']]    ax.bar(['Min', 'Avg', 'Max'], data, color=['red', 'green', 'blue'])    ax.set_ylabel('Lifetime (slots)')    ax.set_title('Node Lifetime')    ax.grid(True, alpha=0.3)        # Delay    ax = axes[0, 1]    ax.bar(['Avg', 'Max'], [metrics['avg_delay'], metrics['max_delay']],            color=['orange', 'red'])    ax.set_ylabel('Delay (slots)')    ax.set_title('Packet Delay')    ax.grid(True, alpha=0.3)        # Throughput    ax = axes[0, 2]    ax.bar(['Total', 'Per Node'],            [metrics['total_throughput'], metrics['avg_node_throughput']])    ax.set_ylabel('Throughput (packets/slot)')    ax.set_title('Throughput')    ax.grid(True, alpha=0.3)        # Transmissions (pie)    ax = axes[1, 0]    ax.pie([metrics['total_packets_sent'], metrics['total_collisions']],            labels=['Success', 'Collision'], autopct='%1.1f%%',           colors=['green', 'red'], startangle=90)    ax.set_title('Transmissions')        # Energy (pie)    ax = axes[1, 1]    consumed = metrics['energy_consumed_ratio'] * 100    ax.pie([consumed, 100-consumed], labels=['Used', 'Remaining'],           autopct='%1.1f%%', colors=['orange', 'blue'], startangle=90)    ax.set_title('Energy')        # Delivery ratio    ax = axes[1, 2]    delivered = metrics['packet_delivery_ratio'] * 100    bars = ax.bar(['Delivered', 'Buffered'], [delivered, 100-delivered],                   color=['green', 'yellow'])    ax.set_ylabel('%')    ax.set_title('Packet Delivery')    ax.set_ylim([0, 105])    ax.grid(True, alpha=0.3)        for bar in bars:        h = bar.get_height()        ax.text(bar.get_x()+bar.get_width()/2, h, f'{h:.1f}%',                ha='center', va='bottom')        plt.tight_layout()    plt.show()print("Visualization functions defined")

## 5. Run SimulationExecute the simulation with configured parameters.

In [ ]:
print("Running simulation...")env = simpy.Environment()sim = Simulator(    env=env,    n_nodes=N_NODES,    lambda_arrival=LAMBDA_ARRIVAL,    q_transmit=Q_TRANSMIT,    ts_idle=TS_IDLE,    tw_wakeup=TW_WAKEUP,    E_initial=E_INITIAL,    PS=PS_SLEEP,    PW=PW_WAKEUP,    PT=PT_TRANSMIT,    PB=PB_BUSY,    simulation_time=SIMULATION_TIME,    seed=RANDOM_SEED)sim.run()metrics = sim.collect_metrics()print("Simulation complete!")

## 6. Results

In [ ]:
print("="*70)print("SIMULATION RESULTS")print("="*70)print(f"\nLifetime:")print(f"  Average: {metrics['avg_lifetime']:.2f} slots")print(f"  Range: [{metrics['min_lifetime']:.0f}, {metrics['max_lifetime']:.0f}]")print(f"\nDelay:")print(f"  Average: {metrics['avg_delay']:.2f} slots")print(f"  Maximum: {metrics['max_delay']:.2f} slots")print(f"\nThroughput:")print(f"  Total: {metrics['total_throughput']:.4f} packets/slot")print(f"  Per node: {metrics['avg_node_throughput']:.4f} packets/slot")print(f"\nTransmissions:")print(f"  Sent: {metrics['total_packets_sent']}")print(f"  Arrived: {metrics['total_packets_arrived']}")print(f"  Delivery ratio: {metrics['packet_delivery_ratio']:.1%}")print(f"  Collisions: {metrics['total_collisions']}")print(f"  Collision rate: {metrics['collision_rate']:.1%}")print(f"\nEnergy:")print(f"  Remaining: {metrics['avg_energy_remaining']:.2f}")print(f"  Consumed: {metrics['energy_consumed_ratio']:.1%}")load = N_NODES * LAMBDA_ARRIVAL * Q_TRANSMITprint(f"\nNetwork Load: {load:.4f} (Aloha capacity: 0.368)")print("="*70)

In [ ]:
# Visualizeplot_results(metrics)

## 7. Parameter Sweep (Optional)Explore how q affects performance.

In [ ]:
# Sweep q from 0.01 to 0.2q_values = np.linspace(0.01, 0.2, 10)results = []print(f"Sweeping {len(q_values)} q values...")for q in q_values:    env = simpy.Environment()    sim = Simulator(env, N_NODES, LAMBDA_ARRIVAL, q, TS_IDLE, TW_WAKEUP,                    E_INITIAL, PS_SLEEP, PW_WAKEUP, PT_TRANSMIT, PB_BUSY,                    3000, RANDOM_SEED)  # Shorter sim for speed    sim.run()    results.append(sim.collect_metrics())print("Sweep complete!")

In [ ]:
# Plot sweep resultsfig, axes = plt.subplots(2, 2, figsize=(12, 10))fig.suptitle('Parameter Sweep: q', fontsize=14, fontweight='bold')q_vals = [r['q_transmit'] for r in results]# Lifetimeax = axes[0, 0]lifetimes = [r['avg_lifetime'] for r in results]ax.plot(q_vals, lifetimes, 'o-', linewidth=2)ax.plot(q_vals[np.argmax(lifetimes)], max(lifetimes), 'r*', markersize=15)ax.set_xlabel('q')ax.set_ylabel('Lifetime (slots)')ax.set_title('Lifetime vs q')ax.grid(True, alpha=0.3)# Delayax = axes[0, 1]delays = [r['avg_delay'] for r in results]ax.plot(q_vals, delays, 'o-', linewidth=2, color='orange')ax.plot(q_vals[np.argmin(delays)], min(delays), 'r*', markersize=15)ax.set_xlabel('q')ax.set_ylabel('Delay (slots)')ax.set_title('Delay vs q')ax.grid(True, alpha=0.3)# Throughputax = axes[1, 0]throughputs = [r['total_throughput'] for r in results]ax.plot(q_vals, throughputs, 'o-', linewidth=2, color='green')ax.plot(q_vals[np.argmax(throughputs)], max(throughputs), 'r*', markersize=15)ax.set_xlabel('q')ax.set_ylabel('Throughput')ax.set_title('Throughput vs q')ax.grid(True, alpha=0.3)# Collision rateax = axes[1, 1]coll_rates = [r['collision_rate'] for r in results]ax.plot(q_vals, coll_rates, 'o-', linewidth=2, color='red')ax.set_xlabel('q')ax.set_ylabel('Collision Rate')ax.set_title('Collision Rate vs q')ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()# Show optimal valuesbest_life_idx = np.argmax(lifetimes)best_delay_idx = np.argmin(delays)print("\nOptimal Values:")print(f"  For max lifetime: q = {q_vals[best_life_idx]:.4f}")print(f"  For min delay: q = {q_vals[best_delay_idx]:.4f}")

## 8. Conclusions### Key Findings1. **On-demand sleep** saves significant energy compared to always-active2. **Transmission probability (q)** affects lifetime and delay3. **Idle timeout (ts)** controls lifetime vs delay tradeoff4. **Network load** should stay below 0.368 (Aloha capacity)### Parameters Effect- **Higher q**: Faster transmission, more collisions- **Lower q**: Fewer collisions, longer delays- **Higher ts**: Better delay, worse lifetime- **Lower ts**: Better lifetime, worse delay### ApplicationsThis simulator can optimize:- IoT network parameters- M2M communication protocols- Energy-constrained systems- Massive device deployments---**Thank you for reviewing this simulator!**For more details, see the complete documentation in the project files.